In [1]:
import pandas as pd
from textblob import TextBlob
import csv
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import os
import numpy as np

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

In [3]:
n = 55
t = 'wizard_test'
s = 32

In [4]:
novel = strip_headers(load_etext(n))
novel = novel.replace('\n', ' ')
novel= TextBlob(novel)
novel_sentences = novel.sentences
novel_title = t

In [6]:
for l in range(2):
    novelWriter = csv.writer(open('novel_'+novel_title+'3.csv', 'w'), delimiter=',')
    for sentence in novel_sentences:
        novelWriter.writerow([sentence])

In [7]:
df_novel = pd.read_csv('novel_'+novel_title+'3.csv', skiprows = s, header=None)

In [8]:
df_novel.head()

,0
0,The Cyclone Dorothy lived in the midst of th...
1,"Their house was small, for the lumber to build..."
2,"There were four walls, a floor and a roof, whi..."
3,Uncle Henry and Aunt Em had a big bed in one c...
4,"There was no garret at all, and no cellar--exc..."


In [9]:
wrd_length = []
total_char = []
def wrd_char_counts(sentence):
    total_chars = 0
    wrd_counts = []
    for word in sentence:
        char_count = len(word)
        wrd_counts.append(char_count)
        total_chars += char_count
    total_char.append(total_chars)
    wrd_length.append(wrd_counts)

In [10]:
for l in df_novel[0]:
    sent = TextBlob(l)
    wrd_char_counts(sent.words)

In [11]:
df_novel['wrd_length'] = wrd_length
df_novel['total_char'] = total_char

In [12]:
df_novel.head()

,0,wrd_length,total_char
0,The Cyclone Dorothy lived in the midst of th...,"[3, 7, 7, 5, 2, 3, 5, 2, 3, 5, 6, 8, 4, 5, 5, ...",113
1,"Their house was small, for the lumber to build...","[5, 5, 3, 5, 3, 3, 6, 2, 5, 2, 3, 2, 2, 7, 2, ...",69
2,"There were four walls, a floor and a roof, whi...","[5, 4, 4, 5, 1, 5, 3, 1, 4, 5, 4, 3, 4, 3, 4, ...",144
3,Uncle Henry and Aunt Em had a big bed in one c...,"[5, 5, 3, 4, 2, 3, 1, 3, 3, 2, 3, 6, 3, 7, 1, ...",75
4,"There was no garret at all, and no cellar--exc...","[5, 3, 2, 6, 2, 3, 3, 2, 6, 6, 1, 5, 4, 3, 2, ...",178


In [13]:
def CountSyllables(word, isName=True):
    vowels = "aeiouy"
    #single syllables in words like bread and lead, but split in names like Breanne and Adreann
    specials = ["ia","ea"] if isName else ["ia"]
    specials_except_end = ["ie","ya","es","ed"]  #seperate syllables unless ending the word
    currentWord = word.lower()
    numVowels = 0
    lastWasVowel = False
    last_letter = ""

    for letter in currentWord:
        if letter in vowels:
            #don't count diphthongs unless special cases
            combo = last_letter+letter
            if lastWasVowel and combo not in specials and combo not in specials_except_end:
                lastWasVowel = True
            else:
                numVowels += 1
                lastWasVowel = True
        else:
            lastWasVowel = False

        last_letter = letter

    #remove es & ed which are usually silent
    if len(currentWord) > 2 and currentWord[-2:] in specials_except_end:
        numVowels -= 1

    #remove silent single e, but not ee since it counted it before and we should be correct
    elif len(currentWord) > 2 and currentWord[-1:] == "e" and currentWord[-2:] != "ee" and currentWord != 'the':
        numVowels -= 1

    return numVowels

In [14]:
syl = []
for l in df_novel[0]:
    sent = TextBlob(l)
    syl_single = []
    for x in sent.words:
        m = CountSyllables(x)
        syl_single.append(m)
    syl.append(syl_single)    

In [15]:
syl_count_arr = []
for n in syl:
    n = np.array(n)
    syl_count_arr.append(n)

In [16]:
df_novel['syl_count'] = syl_count_arr

In [17]:
df_novel.head()

,0,wrd_length,total_char,syl_count
0,The Cyclone Dorothy lived in the midst of th...,"[3, 7, 7, 5, 2, 3, 5, 2, 3, 5, 6, 8, 4, 5, 5, ...",113,"[1, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, ..."
1,"Their house was small, for the lumber to build...","[5, 5, 3, 5, 3, 3, 6, 2, 5, 2, 3, 2, 2, 7, 2, ...",69,"[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, ..."
2,"There were four walls, a floor and a roof, whi...","[5, 4, 4, 5, 1, 5, 3, 1, 4, 5, 4, 3, 4, 3, 4, ...",144,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Uncle Henry and Aunt Em had a big bed in one c...,"[5, 5, 3, 4, 2, 3, 1, 3, 3, 2, 3, 6, 3, 7, 1, ...",75,"[1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 3, 1, ..."
4,"There was no garret at all, and no cellar--exc...","[5, 3, 2, 6, 2, 3, 3, 2, 6, 6, 1, 5, 4, 3, 2, ...",178,"[1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, ..."


In [18]:
syl_sum = []
for l in range(0,len(df_novel)):
    syl_sum.append(df_novel['syl_count'][l].sum())

In [19]:
df_novel['syl_sum'] = syl_sum

In [20]:
def detect_sentiment(text):
    return TextBlob(text.decode('utf-8')).sentiment.polarity

In [21]:
df_novel['sentiment'] = df_novel[0].apply(detect_sentiment)

In [23]:
df_novel.to_csv('novel_'+novel_title+'3_2.csv', index=False)

In [24]:
#test
df_test = pd.read_csv('novel_'+novel_title+'3_2.csv')

In [25]:
df_test.head()

,0,wrd_length,total_char,syl_count,syl_sum,sentiment
0,The Cyclone Dorothy lived in the midst of th...,"[3, 7, 7, 5, 2, 3, 5, 2, 3, 5, 6, 8, 4, 5, 5, ...",113,[1 2 3 1 1 1 1 1 1 2 2 2 1 1 2 1 1 1 2 1 1 1 1...,36,0.80000
1,"Their house was small, for the lumber to build...","[5, 5, 3, 5, 3, 3, 6, 2, 5, 2, 3, 2, 2, 7, 2, ...",69,[1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 2 2 1],22,0.12500
2,"There were four walls, a floor and a roof, whi...","[5, 4, 4, 5, 1, 5, 3, 1, 4, 5, 4, 3, 4, 3, 4, ...",144,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 1 2...,40,0.00000
3,Uncle Henry and Aunt Em had a big bed in one c...,"[5, 5, 3, 4, 2, 3, 1, 3, 3, 2, 3, 6, 3, 7, 1, ...",75,[1 2 1 1 1 1 1 1 0 1 1 2 1 3 1 1 0 1 3 2],25,-0.09375
4,"There was no garret at all, and no cellar--exc...","[5, 3, 2, 6, 2, 3, 3, 2, 6, 6, 1, 5, 4, 3, 2, ...",178,[1 1 1 2 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 2 1 1...,57,0.23750


In [26]:
df_cluster = df_novel.drop('wrd_length', 1)

In [27]:
df_cluster = df_cluster.drop('syl_count', 1)

In [28]:
df_cluster.head()

,0,total_char,syl_sum,sentiment
0,The Cyclone Dorothy lived in the midst of th...,113,36,0.80000
1,"Their house was small, for the lumber to build...",69,22,0.12500
2,"There were four walls, a floor and a roof, whi...",144,40,0.00000
3,Uncle Henry and Aunt Em had a big bed in one c...,75,25,-0.09375
4,"There was no garret at all, and no cellar--exc...",178,57,0.23750


In [29]:
X = df_cluster.drop(0, axis=1)

In [30]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state=1)
km.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=1, tol=0.0001,
    verbose=0)

In [31]:
km.labels_

array([2, 2, 1, ..., 0, 0, 0], dtype=int32)

In [32]:
df_cluster['cluster'] = km.labels_

In [33]:
km.cluster_centers_

array([[  3.09553232e+01,   9.77471483e+00,   4.44280815e-02],
       [  1.55976510e+02,   4.80637584e+01,   1.31426539e-01],
       [  8.81676040e+01,   2.76074241e+01,   1.08692888e-01]])

In [34]:
df_cluster.groupby('cluster').mean()

,total_char,syl_sum,sentiment
cluster,,,
0,30.955323,9.774715,0.044428
1,155.976510,48.063758,0.131427
2,88.167604,27.607424,0.108693


In [35]:
df_cluster.head()

,0,total_char,syl_sum,sentiment,cluster
0,The Cyclone Dorothy lived in the midst of th...,113,36,0.80000,2
1,"Their house was small, for the lumber to build...",69,22,0.12500,2
2,"There were four walls, a floor and a roof, whi...",144,40,0.00000,1
3,Uncle Henry and Aunt Em had a big bed in one c...,75,25,-0.09375,2
4,"There was no garret at all, and no cellar--exc...",178,57,0.23750,1


In [36]:
df_cluster_char = df_cluster.drop('syl_sum', 1)

In [38]:
X = df_cluster_char.drop(0, axis=1)

In [39]:
df_cluster_char.head()

,0,total_char,sentiment,cluster
0,The Cyclone Dorothy lived in the midst of th...,113,0.80000,2
1,"Their house was small, for the lumber to build...",69,0.12500,2
2,"There were four walls, a floor and a roof, whi...",144,0.00000,1
3,Uncle Henry and Aunt Em had a big bed in one c...,75,-0.09375,2
4,"There was no garret at all, and no cellar--exc...",178,0.23750,1


In [41]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [42]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [43]:
km = KMeans(n_clusters=3, random_state=1)
km.fit(X_scaled)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=1, tol=0.0001,
    verbose=0)

In [44]:
km.labels_

array([0, 1, 1, ..., 2, 2, 0], dtype=int32)

In [45]:
df_cluster_char['cluster'] = km.labels_

In [46]:
df_cluster_char.groupby('cluster').mean()

,total_char,sentiment
cluster,,
0,87.808511,0.571014
1,105.932620,0.002781
2,30.247179,-0.008135


In [47]:
df_cluster5 = df_cluster

In [48]:
df_cluster5.head()

,0,total_char,syl_sum,sentiment,cluster
0,The Cyclone Dorothy lived in the midst of th...,113,36,0.80000,2
1,"Their house was small, for the lumber to build...",69,22,0.12500,2
2,"There were four walls, a floor and a roof, whi...",144,40,0.00000,1
3,Uncle Henry and Aunt Em had a big bed in one c...,75,25,-0.09375,2
4,"There was no garret at all, and no cellar--exc...",178,57,0.23750,1


In [51]:
X = df_cluster5.drop(0, axis=1)

In [52]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [53]:
km = KMeans(n_clusters=5, random_state=1)
km.fit(X_scaled)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=1, tol=0.0001,
    verbose=0)

In [54]:
km.labels_

array([3, 2, 0, ..., 1, 1, 4], dtype=int32)

In [55]:
df_cluster5['cluster'] = km.labels_

In [56]:
df_cluster5.groupby('cluster').mean()

,total_char,syl_sum,sentiment
cluster,,,
0,157.402827,48.459364,0.103457
1,29.578658,9.297030,-0.034261
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,39.706294,12.811189,0.544628


In [57]:
df_5_clustered = df_cluster5.groupby('cluster').mean()

In [58]:
df_5_clustered.head()

,total_char,syl_sum,sentiment
cluster,,,
0,157.402827,48.459364,0.103457
1,29.578658,9.297030,-0.034261
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,39.706294,12.811189,0.544628


In [59]:
c = df_5_clustered.groupby('sentiment').min()

In [60]:
c

,total_char,syl_sum
sentiment,,
-0.040968,87.723748,27.253635
-0.034261,29.578658,9.297030
0.103457,157.402827,48.459364
0.462716,91.284211,29.059649
0.544628,39.706294,12.811189


In [71]:
c2 = df_5_clustered.groupby('total_char').min()

In [73]:
c2

,syl_sum,sentiment
total_char,,
29.578658,9.297030,-0.034261
39.706294,12.811189,0.544628
87.723748,27.253635,-0.040968
91.284211,29.059649,0.462716
157.402827,48.459364,0.103457


In [75]:
type(c2)

pandas.core.frame.DataFrame

In [82]:
t = c2['sentiment']

In [86]:

c3 = c2.set_index([range(5)])

In [87]:
keys = 
c3

,syl_sum,sentiment
0,9.297030,-0.034261
1,12.811189,0.544628
2,27.253635,-0.040968
3,29.059649,0.462716
4,48.459364,0.103457


In [88]:
c4 = c2.reset_index(level=0, inplace=True)

In [89]:
c4

In [90]:
c2

,total_char,syl_sum,sentiment
0,29.578658,9.297030,-0.034261
1,39.706294,12.811189,0.544628
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,157.402827,48.459364,0.103457


In [91]:
s = df_5_clustered.groupby('sentiment').min()

In [92]:
s

,total_char,syl_sum
sentiment,,
-0.040968,87.723748,27.253635
-0.034261,29.578658,9.297030
0.103457,157.402827,48.459364
0.462716,91.284211,29.059649
0.544628,39.706294,12.811189


In [93]:
s.reset_index(level=0, inplace=True)

In [94]:
s

,sentiment,total_char,syl_sum
0,-0.040968,87.723748,27.253635
1,-0.034261,29.578658,9.297030
2,0.103457,157.402827,48.459364
3,0.462716,91.284211,29.059649
4,0.544628,39.706294,12.811189


In [95]:
c = c2

In [100]:
c2

,total_char,syl_sum,sentiment
0,29.578658,9.297030,-0.034261
1,39.706294,12.811189,0.544628
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,157.402827,48.459364,0.103457


In [96]:
c

,total_char,syl_sum,sentiment
0,29.578658,9.297030,-0.034261
1,39.706294,12.811189,0.544628
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,157.402827,48.459364,0.103457


In [99]:
c['total_char'][0]

29.578657865786578

In [ ]:
c.query(c)

In [103]:
for x in range(0,5):
    print x

0
1
2
3
4


In [107]:
matches = []
for x in range(0,5):
    if (c['total_char'][x] == c2['total_char'][1]) & (c['sentiment'][x] == c2['sentiment'][1]):
        matches.append(1)
    else:
        matches.append(0)

In [108]:
matches

[0, 1, 0, 0, 0]

In [ ]:
c2_c_matches = []

c2_c_match = []

In [111]:
matches = []
for x in range(0,5):
    if (c['total_char'][x] == c2['total_char'][1]) & (c['sentiment'][x] == c2['sentiment'][1]):
        matches.append(1)
        print c['total_char'][x]
        print c['sentiment'][x]
        print c.index
    else:
        matches.append(0)

39.7062937063
0.544628427128
RangeIndex(start=0, stop=5, step=1)


In [112]:
matches

[0, 1, 0, 0, 0]

In [114]:
matches = []
for x in range(0,5):
    if (c['total_char'][x] == c2['total_char'][2]) & (c['sentiment'][x] == c2['sentiment'][2]):
        matches.append(1)
        print c['total_char'][x]
        print c['sentiment'][x]
        print x
    else:
        matches.append(0)

87.7237479806
-0.040968442794
2


In [115]:
matches

[0, 0, 1, 0, 0]

In [120]:
c2_match = np.zeros(5)

In [121]:
c2_match

array([ 0.,  0.,  0.,  0.,  0.])

In [122]:
c['c2_match'] = c2_match

In [130]:
c

,total_char,syl_sum,sentiment,c2_match,c_match
0,29.578658,9.297030,-0.034261,0.0,0.0
1,39.706294,12.811189,0.544628,0.0,0.0
2,87.723748,27.253635,-0.040968,0.0,0.0
3,91.284211,29.059649,0.462716,0.0,0.0
4,157.402827,48.459364,0.103457,0.0,0.0


In [126]:
c_match = np.zeros(5)

In [127]:
c_match

array([ 0.,  0.,  0.,  0.,  0.])

In [128]:
c2['c_match'] = c_match

In [129]:
c2

,total_char,syl_sum,sentiment,c2_match,c_match
0,29.578658,9.297030,-0.034261,0.0,0.0
1,39.706294,12.811189,0.544628,0.0,0.0
2,87.723748,27.253635,-0.040968,0.0,0.0
3,91.284211,29.059649,0.462716,0.0,0.0
4,157.402827,48.459364,0.103457,0.0,0.0


In [135]:
c2['c_match'][1] = 1

In [136]:
c

,total_char,syl_sum,sentiment,c2_match,c_match
0,29.578658,9.297030,-0.034261,0.0,0.0
1,39.706294,12.811189,0.544628,0.0,1.0
2,87.723748,27.253635,-0.040968,0.0,0.0
3,91.284211,29.059649,0.462716,0.0,0.0
4,157.402827,48.459364,0.103457,0.0,0.0


In [139]:
alice_cluster_5 = pd.read_csv('alice_cluster_5.csv')

In [140]:
alice_cluster_5

,total_char,syl_sum,sentiment
0,37.585311,11.528814,-0.065021
1,60.933333,18.894444,0.492801
2,529.555556,161.666667,0.009813
3,169.704225,52.105634,0.033182
4,46.026119,14.119403,0.002250


In [141]:
wizard_cluster_5 = df_cluster5.groupby('cluster').mean()

In [142]:
wizard_cluster_5

,total_char,syl_sum,sentiment
cluster,,,
0,157.402827,48.459364,0.103457
1,29.578658,9.297030,-0.034261
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,39.706294,12.811189,0.544628


In [143]:
wizard_cluster_5.to_csv('wizard_cluster_5.csv', index=False)

In [144]:
wizard_cluster_5 = pd.read_csv('wizard_cluster_5.csv')

In [145]:
wizard_cluster_5

,total_char,syl_sum,sentiment
0,157.402827,48.459364,0.103457
1,29.578658,9.297030,-0.034261
2,87.723748,27.253635,-0.040968
3,91.284211,29.059649,0.462716
4,39.706294,12.811189,0.544628


In [148]:
wizard_cluster_5['total_char'][0]

157.402826855

In [ ]:
matches = []
for x in range(0,5):
    if (c['total_char'][x] == c2['total_char'][2]) & (c['sentiment'][x] == c2['sentiment'][2]):
        matches.append(1)
        print c['total_char'][x]
        print c['sentiment'][x]
        print x
    else:
        matches.append(0)